# Using OpenAI models from Azure AI

You can also use an OpenAI model deployed into Azure AI.
For this, you will provide a few pieces of information from the Azure AI playground:

In [ ]:
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# This is the name of the model deployed, such as 'gpt-4' or 'gpt-35-turbo
model = 'gpt-4'
# This is the deployment URL, as provided in the Azure AI playground ('view code'):
azure_endpoint = os.getenv("AZUREAI_ENDPOINT", "Please set the endpoint")
# The environment variable should be set to the API key from the Azure AI playground:
api_key=os.getenv("AZUREAI_KEY", "Please set API key")
# Alternatively, we can use Entra authentication
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

We can now construct the `guidance` model object:

In [ ]:
from guidance import models, gen

azureai_model = models.AzureOpenAIChat(
    model=model,
    azure_endpoint=azure_endpoint,
    # For authentication, use either
    azure_ad_token_provider=token_provider,
    # or
    # api_key=api_key
)

We can use the model as before:

In [ ]:
from guidance import system, user, assistant

with system():
    lm = azureai_model + "You are a helpful assistant."
    
with user():
    lm += "What is the meaning of life?"

with assistant():
    lm += gen("response")

This works with the multistep example as well:

In [ ]:
import guidance
# you can create and guide multi-turn conversations by using a series of role tags
@guidance
def experts(lm, query):
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""\
        I want a response to the following question:
        {query}
        Who are 3 world-class experts (past or present) who would be great at answering this?
        Please don't answer the question or comment on it yet."""

    with assistant():
        lm += gen(name='experts', max_tokens=300)
    
    with user():
        lm += f"""\
        Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
        In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
        If the experts would disagree, just present their different positions as alternatives in the answer itself
        (e.g. 'some might argue... others might argue...').
        Please start your answer with ANSWER:"""
    
    with assistant():
        lm += gen(name='answer', max_tokens=500)

    return lm
                   
azureai_model + experts(query='What is the meaning of life?')